In [ ]:
# ライブラリのDL(termextract)
!wget http://gensen.dl.itc.u-tokyo.ac.jp/soft/pytermextract-0_02.zip 

In [ ]:
# ライブラリの解凍
import shutil
shutil.unpack_archive('pytermextract-0_02.zip', 'pytermextract')

In [ ]:
# ライブラリのインストール
# cd /pytermextract/
# pytermextract> pip install .

In [ ]:
# 1.スクレイピングしたhtmlから平文を抽出
def get_body(url):
    from bs4 import BeautifulSoup #必要なモジュールのインポート
    import requests
    
    try:
        html = requests.get(url) #Webページの取得
    except:
        print("Invalid url!") #Webページの取得に失敗した場合の処理
        return -1
    soup = BeautifulSoup(html.content, "html.parser") #Webページからhtmlの取得
    return soup.find("body").text #htmlからbodyを取り出し、テキストのみを返す

In [ ]:
# 関数get_bodyの結果表示
print(get_body("https://www.softbank.jp/biz/ai/"))

In [ ]:
# 2.キーワード抽出（Janome利用）
def extract_keyword(text):
    import termextract.janome #必要なモジュールのインポート
    import termextract.core
    from janome.tokenizer import Tokenizer
    import collections 
    
    t = Tokenizer()
    tokenize_text = t.tokenize(text) #janomeによる日本語の形態素解析の処理
    frequency = termextract.janome.cmp_noun_dict(tokenize_text) #複合語抽出処理

    lr = termextract.core.score_lr( #FrequencyからLRを生成する
        frequency,
        ignore_words=termextract.mecab.IGNORE_WORDS,
        lr_mode=1, average_rate=1)
    
    term_imp = termextract.core.term_importance(frequency, lr) #FrequencyとLRを組み合わせ、FLRの重要度を出す

    data_collection = collections.Counter(term_imp) #collectionsを使用して重要度が高い順に整理
    return data_collection.most_common()[0][0] #もっとも重要度の高い単語を返す

In [ ]:
# 関数extract_keywordの結果表示
extract_keyword(get_body("https://www.softbank.jp/biz/ai/"))

In [ ]:
# 3.google検索の結果からスクレイピングと解析を実施
def search_web(keyword):
    import requests #必要なモジュールのインポート
    from bs4 import BeautifulSoup
    import re
    from janome.analyzer import Analyzer
    from janome.tokenfilter import POSKeepFilter, TokenCountFilter
    

    search_url = 'https://www.google.co.jp/search?hl=ja&num=20&q=' + keyword #検索ワードの設定
    res_google = requests.get(search_url) #検索の実行
    
    bs4_google = BeautifulSoup(res_google.text, 'html.parser') #検索結果をhtmlにして取得
    search_results = bs4_google.select('.kCrYT > a') #htmlから検索結果一覧の部分を取得(kCrYTは変更されてる可能性あり)
 
    max = 0 #キーワードの出現回数のカウントのための変数

    for search_result in search_results:

        page_url = re.sub("\/url\?q=","",search_result.get('href')) #urlから余計な文字列の削除
        page_url = re.sub("&sa=U&ved.*", "", page_url)


        html = requests.get(page_url) #urlを開いてWebページ情報を取得
        soup_page = BeautifulSoup(html.content, "html.parser") #Webページからhtmlを取得
    
        token_filters = [
        POSKeepFilter(['名詞']),  # 名詞を抽出するようにする
        TokenCountFilter(),  # トークンの出現回数をカウントする
        ]
        analyzer = Analyzer(token_filters=token_filters)  # フィルターからアナライザーを生成
    
        try:
            word_count = analyzer.analyze(soup_page.find("body").text) #それぞれの名詞の出現回数のカウント
        except NameError:
            continue #スクレイピング対策などによりページを開けない場合はcontinue
    
        try:
            num = dict(word_count)[keyword] #キーワードの出現回数の取得
            if max < int(num): #キーワードの出現回数が、今までの最大値を上回っている場合
                max = num #出現回数の最大値を更新
                url = page_url #urlの更新
                title = search_result.text #タイトルの更新
        except KeyError:
            continue #キーワードがない場合はcontinue
        
    return max,title, url #キーワードの出現回数、タイトル、urlを返す

In [ ]:
# 関数を総括する

url = "https://www.softbank.jp/biz/ai/" #スクレイピングするURL

body = get_body(url) #bodyの抜き出し
keyword = extract_keyword(body) #bodyからキーワードの抜き出し
result = search_web(keyword) #キーワードから検索、キーワード出現回数のカウント

print("検索キーワード:", keyword)
print("出現回数:", result[0])
print("タイトル:", result[1])
print("URL:", result[2])